# Begin By Importing Necescary Packages

In [47]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# Load Datasets and Connect to get Input for SVR

In [54]:
arrest_data = pd.read_csv("arrests_w_census_loc.csv")
arrest_data.PERP_RACE[arrest_data.PERP_RACE.str.contains("WHITE")]=0
arrest_data.PERP_RACE[arrest_data.PERP_RACE != 0]=1
arrest_data = arrest_data.groupby(["PERP_RACE","BlockLocation"]).size().reset_index(name='counts')
blockLocation = arrest_data["BlockLocation"]
blockLat = [float(re.findall(r'[-\d\.]+', bl)[0]) for bl in blockLocation]
blockLon = [float(re.findall(r'[-\d\.]+', bl)[1]) for bl in blockLocation]
arrest_data["blockLat"]=blockLat
arrest_data["blockLon"]=blockLon
arrest_data = arrest_data.drop("BlockLocation", axis=1)
arrest_data = arrest_data.rename(columns={"counts": "Num_Arrests", "PERP_RACE": "Race"})
block_data = pd.read_csv("census_block_loc.csv")
block_data = pd.merge(left=arrest_data, right=block_data,
                      left_on=["blockLat","blockLon"], right_on=["Latitude","Longitude"])
census_data = pd.read_csv("nyc_census_tracts.csv")
tracts = block_data["BlockCode"]
tracts = [int(str(tract)[:-4]) for tract in tracts]
block_data["tracts"]=tracts
block_data = block_data.drop(columns=["Latitude","Longitude","BlockCode","County","blockLat","blockLon"])
data = pd.merge(left=block_data, right=census_data, left_on="tracts", right_on="CensusTract")
data = data.drop("tracts", axis=1)
data = data.drop("State", axis=1)
data

/home/erik7brown/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/erik7brown/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Race,Num_Arrests,CensusTract,County,Borough,TotalPop,Men,Women,Hispanic,White,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,0,1,36085024402,Richmond,Staten Island,4241,2023,2218,3.7,84.5,...,1.1,0.6,4.0,44.3,2046,75.2,21.2,3.6,0.0,8.3
1,0,4,36085024402,Richmond,Staten Island,4241,2023,2218,3.7,84.5,...,1.1,0.6,4.0,44.3,2046,75.2,21.2,3.6,0.0,8.3
2,0,1,36085024402,Richmond,Staten Island,4241,2023,2218,3.7,84.5,...,1.1,0.6,4.0,44.3,2046,75.2,21.2,3.6,0.0,8.3
3,0,1,36085024402,Richmond,Staten Island,4241,2023,2218,3.7,84.5,...,1.1,0.6,4.0,44.3,2046,75.2,21.2,3.6,0.0,8.3
4,0,2,36085024402,Richmond,Staten Island,4241,2023,2218,3.7,84.5,...,1.1,0.6,4.0,44.3,2046,75.2,21.2,3.6,0.0,8.3
5,1,1,36085024402,Richmond,Staten Island,4241,2023,2218,3.7,84.5,...,1.1,0.6,4.0,44.3,2046,75.2,21.2,3.6,0.0,8.3
6,0,3,36085024401,Richmond,Staten Island,6408,2755,3653,8.3,87.5,...,0.1,0.0,3.9,44.6,2703,70.2,29.3,0.6,0.0,6.6
7,0,3,36085024401,Richmond,Staten Island,6408,2755,3653,8.3,87.5,...,0.1,0.0,3.9,44.6,2703,70.2,29.3,0.6,0.0,6.6
8,1,2,36085024401,Richmond,Staten Island,6408,2755,3653,8.3,87.5,...,0.1,0.0,3.9,44.6,2703,70.2,29.3,0.6,0.0,6.6
9,0,4,36085024401,Richmond,Staten Island,6408,2755,3653,8.3,87.5,...,0.1,0.0,3.9,44.6,2703,70.2,29.3,0.6,0.0,6.6


# Merge Data by County to Smooth Block-level Arrests

In [81]:
raw_data = pd.DataFrame(data)
total_pops = dict(raw_data.groupby("County").sum()["TotalPop"])
Bronx_data = raw_data.loc[raw_data.County == "Bronx"]
Kings_data = raw_data.loc[raw_data.County == "Kings"]
New_York_data = raw_data.loc[raw_data.County == "New York"]
Queens_data = raw_data.loc[raw_data.County == "Queens"]
Richmond_data = raw_data.loc[raw_data.County == "Richmond"]
data_dict = {"Bronx":Bronx_data, "Kings":Kings_data, "New York":New_York_data, "Queens":Queens_data, "Richmond":Richmond_data}
for county, dataset in data_dict.items():
    data_dict[county] = dataset.drop(["County","Borough"], axis=1)


,Race,Num_Arrests,CensusTract,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
8270,0,4,36005000100,7703,7133,570,29.9,6.1,60.9,0.2,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
8271,1,2,36005000100,7703,7133,570,29.9,6.1,60.9,0.2,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
8272,0,1,36005000100,7703,7133,570,29.9,6.1,60.9,0.2,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
8273,1,1,36005000100,7703,7133,570,29.9,6.1,60.9,0.2,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
8388,0,1,36005001900,2591,1206,1385,55.4,9.0,29.0,0.0,...,3.0,2.4,6.2,45.4,1083,76.8,15.5,7.7,0.0,19.2
8389,0,2,36005001900,2591,1206,1385,55.4,9.0,29.0,0.0,...,3.0,2.4,6.2,45.4,1083,76.8,15.5,7.7,0.0,19.2
8390,1,1,36005001900,2591,1206,1385,55.4,9.0,29.0,0.0,...,3.0,2.4,6.2,45.4,1083,76.8,15.5,7.7,0.0,19.2
8391,0,4,36005001900,2591,1206,1385,55.4,9.0,29.0,0.0,...,3.0,2.4,6.2,45.4,1083,76.8,15.5,7.7,0.0,19.2
8392,1,10,36005001900,2591,1206,1385,55.4,9.0,29.0,0.0,...,3.0,2.4,6.2,45.4,1083,76.8,15.5,7.7,0.0,19.2
8393,0,1,36005001900,2591,1206,1385,55.4,9.0,29.0,0.0,...,3.0,2.4,6.2,45.4,1083,76.8,15.5,7.7,0.0,19.2


# Apply Feature Scaling to Continuous Features

In [49]:
num_cols = ['TotalPop','Men','Women','Hispanic','White','Black','Native','Asian','Citizen','Income','IncomeErr','IncomePerCap','IncomePerCapErr','Poverty','ChildPoverty','Professional','Service','Office','Construction','Production','Drive','Carpool','Transit','Walk','OtherTransp','WorkAtHome','MeanCommute','Employed','PrivateWork','PublicWork','SelfEmployed','FamilyWork','Unemployment']
X = data[num_cols].values
data.drop(columns=num_cols)
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
num_data = pd.DataFrame(X, columns=num_cols)
data[num_cols]=num_data
data = data.dropna()

# Split Data by Race and Into Train/Test Datasets

In [50]:
cols = data.shape[1]
data_white = data[data["Race"]==0].drop("Race", axis=1)
data_non_white=data[data["Race"]==1].drop("Race", axis=1)
data_white_train = data_white.sample(frac=0.8,random_state=101)
data_white_test  = data_white.drop(data_white_train.index)
data_non_white_train = data_non_white.sample(frac=0.8,random_state=101)
data_non_white_test  = data_non_white.drop(data_non_white_train.index)
Xw = data_white_train.iloc[:,1:cols]
yw = data_white_train.iloc[:,0]
Xnw = data_non_white_train.iloc[:,1:cols]
ynw = data_non_white_train.iloc[:,0]
Xwt = data_white_test.iloc[:,1:cols]
ywt = data_white_test.iloc[:,0]
Xnwt = data_non_white_test.iloc[:,1:cols]
ynwt = data_non_white_test.iloc[:,0]

# Encode Categorical Features as Numerical Values

In [51]:
cat_cols = ['CensusTract','County','Borough']
encoder = LabelEncoder()
for var in cat_cols:
    Xw[var] = encoder.fit_transform(Xw[var])
    Xnw[var] = encoder.fit_transform(Xnw[var])
    Xwt[var] = encoder.fit_transform(Xwt[var])
    Xnwt[var] = encoder.fit_transform(Xnwt[var])

# See If Polynomial or RBF Nonlinear Kernels will Work Better

In [6]:
rbfw = SVR(gamma='scale',kernel='rbf')
cv = cross_val_score(rbfw, Xw, yw, cv=20, scoring="neg_mean_squared_error")
print("The average MSE for White RBF is",-sum(cv)/len(cv))

rbfnw = SVR(gamma='scale',kernel='rbf')
cv = cross_val_score(rbfnw, Xnw, ynw, cv=20, scoring="neg_mean_squared_error")
print("The average MSE for Non-White RBF is",-sum(cv)/len(cv))

polyw = SVR(gamma='scale',kernel='poly')
cv = cross_val_score(polyw, Xw, yw, cv=20, scoring="neg_mean_squared_error")
print("The average MSE for White Polynomial is",-sum(cv)/len(cv))

polynw = SVR(gamma='scale',kernel='poly')
cv = cross_val_score(polynw, Xnw, ynw, cv=20, scoring="neg_mean_squared_error")
print("The average MSE for Non-White Polynomial is",-sum(cv)/len(cv))

The average MSE for White RBF is 5.163233119789454
The average MSE for Non-White RBF is 5.4965227391641545
The average MSE for White Polynomial is 4.979140224178782
The average MSE for Non-White Polynomial is 5.582080213473355


# Randomized Search to Find Optimal HyperParams

In [7]:
params = {"C":uniform(0,10),"epsilon":uniform(0.5,1.5),"kernel":["poly","rbf"]}
#note gamma was not included as it is quite sensitive
rscv = RandomizedSearchCV(SVR(gamma="scale"),params,cv=20,iid=False)
rscv.fit(Xw,yw)
print("optimal C", rscv.best_estimator_.get_params()["C"])
print("optimal epsilon", rscv.best_estimator_.get_params()["epsilon"])
print("optimal kernel", rscv.best_estimator_.get_params()["kernel"])
cv = cross_val_score(rscv, Xw, yw, cv=20, scoring="neg_mean_squared_error")
print("The average MSE for White is",-sum(cv)/len(cv))

rscv2 = RandomizedSearchCV(SVR(gamma="scale"),params,cv=20,iid=False)
rscv2.fit(Xnw,ynw)
print("optimal C", rscv2.best_estimator_.get_params()["C"])
print("optimal epsilon", rscv2.best_estimator_.get_params()["epsilon"])
print("optimal kernel", rscv2.best_estimator_.get_params()["kernel"])
cv = cross_val_score(rscv, Xnw, ynw, cv=20, scoring="neg_mean_squared_error")
print("The average MSE for Non-White is",-sum(cv)/len(cv))


KeyboardInterrupt: 

# Get Test Data Metrics

In [8]:
Final_model = SVR(kernel='rbf', C=9, gamma="scale", epsilon=1.1)
Final_model.fit(Xw, yw)
test_predicted = Final_model.predict(Xwt)
mse = mean_squared_error(ywt, test_predicted)
print("Test MSE for White: ",mse)
mae = mean_absolute_error(ywt, test_predicted)
print("Test MAE for White: ",mae)
Final_model.fit(Xnw, ynw)
test_predicted_2 = Final_model.predict(Xnwt)
mse = mean_squared_error(ynwt, test_predicted_2)
print("Test MSE for Non-White: ",mse)
mae = mean_absolute_error(ynwt, test_predicted_2)
print("Test MAE for Non-White: ",mae)

Test MSE for White:  4.7677110490599475
Test MAE for White:  1.7484821006319535
Test MSE for Non-White:  5.110773986254233
Test MAE for Non-White:  1.871852750044117


# Try Random Forrest Instead

In [9]:
model = RandomForestRegressor()
model.fit(Xw, yw)
cv = cross_val_score(Final_model, Xw, yw, cv=20, scoring="neg_mean_squared_error")
print("The average MSE for White is",-sum(cv)/len(cv))
model.fit(Xnw, ynw)
cv = cross_val_score(Final_model, Xnw, ynw, cv=20, scoring="neg_mean_squared_error")
print("The average MSE for Non-White is",-sum(cv)/len(cv))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


The average MSE for White is 4.688111127681427


KeyboardInterrupt: 

# Try to find Optimal Hyperparams for Random Forrest

In [ ]:
params = {"n_estimators":[10,20,50,100], "min_samples_split":[2,5,10,15,20], "max_features":[5,10,15,20]}

rscv = RandomizedSearchCV(RandomForestRegressor(),params,cv=20,iid=False)
rscv.fit(Xw,yw)
print("optimal n_estimators", rscv.best_estimator_.get_params()["n_estimators"])
print("optimal min_samples_split", rscv.best_estimator_.get_params()["min_samples_split"])
print("optimal max_features", rscv.best_estimator_.get_params()["max_features"])

rscv2 = RandomizedSearchCV(RandomForestRegressor(),params,cv=20,iid=False)
rscv2.fit(Xnw,ynw)
print("optimal n_estimators", rscv2.best_estimator_.get_params()["n_estimators"])
print("optimal min_samples_split", rscv2.best_estimator_.get_params()["min_samples_split"])
print("optimal max_features", rscv2.best_estimator_.get_params()["max_features"])

# Fit Best Model (Random Forrest)

In [ ]:
Final_model = RandomForestRegressor(n_estimators=50, min_samples_split=20, max_features=10)
Final_model.fit(Xw, yw)
test_predicted = Final_model.predict(Xwt)
mse = mean_squared_error(ywt, test_predicted)
print("Test MSE for White: ",mse)
mae = mean_absolute_error(ywt, test_predicted)
print("Test MAE for White: ",mae)
Final_model.fit(Xnw, ynw)
test_predicted_2 = Final_model.predict(Xnwt)
mse = mean_squared_error(ynwt, test_predicted_2)
print("Test MSE for Non-White: ",mse)
mae = mean_absolute_error(ynwt, test_predicted_2)
print("Test MAE for Non-White: ",mae)

# Predict Output And Prepare Bias Metric

In [ ]:
Final_model.fit(Xw, yw)
Xw["predicted_w"] = Final_model.predict(Xw)
Final_model.fit(Xnw, ynw)
Xnw["predicted_nw"] = Final_model.predict(Xnw)
Xnw = Xnw[["CensusTract", "predicted_nw"]]
X = pd.merge(left=Xw, right=Xnw,left_on="CensusTract", right_on="CensusTract")
X["Non-White"]=1-X["White"]
X["y"] = np.log(abs(X["predicted_w"]/X["White"]-X["predicted_nw"]/X["Non-White"]))
X = X.replace(np.inf, np.nan)
X = X.dropna()
X_train = X.sample(frac=0.8,random_state=101)
Xt=X.drop(X_train.index)
X=X_train
y = X["y"]
yt=Xt["y"]
X = X.drop(columns=['Hispanic','Black','Native','Asian',"White","Non-White","y"])
Xt = Xt.drop(columns=['Hispanic','Black','Native','Asian',"White","Non-White","y"])

# See If Polynomial or RBF Nonlinear Kernels will Work Better

In [ ]:
rbf = SVR(gamma='scale',kernel='rbf')
cv = cross_val_score(rbf, X, y, cv=20, scoring="neg_mean_squared_error")
print("The average MSE for RBF is",-sum(cv)/len(cv))

poly = SVR(gamma='scale',kernel='poly')
cv = cross_val_score(poly, X, y, cv=20, scoring="neg_mean_squared_error")
print("The average MSE for Polynomial is",-sum(cv)/len(cv))

# Randomized Search to Find Optimal HyperParams

In [ ]:
params = {"C":uniform(0,10),"gamma":uniform(0.001,1),"epsilon":uniform(0,1)}

rscv = RandomizedSearchCV(SVR(kernel="rbf"),params,cv=20,iid=False)
rscv.fit(Xw,yw)
print("optimal C", rscv.best_estimator_.get_params()["C"])
print("optimal gamma", rscv.best_estimator_.get_params()["gamma"])
print("optimal epsilon", rscv.best_estimator_.get_params()["epsilon"])
cv = cross_val_score(rscv, X, y, cv=20, scoring="neg_mean_squared_error")
print("The average MSE is",-sum(cv)/len(cv))

# Get Test Data Metrics

In [ ]:
Final_model = SVR(kernel='rbf', C=4, gamma=0.15, epsilon=0.15)
Final_model.fit(X, y)
test_predicted = Final_model.predict(Xt)
mse = mean_squared_error(yt, test_predicted)
print("Test MSE: ",mse)
mae = mean_absolute_error(yt, test_predicted)
print("Test MAE: ",mae)